In [24]:
!wget -q https://raw.githubusercontent.com/ml-compling-2022-hse/ml-intro/main/class2/IMDB-Movie-Data.csv
!pip3 -q install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 48.2 MB/s eta 0:00:00


In [100]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [98]:
# импорты необходимых библиотек
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

# import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from string import punctuation
from typing import Callable

In [21]:
data = pd.read_csv('IMDB-Movie-Data.csv')

(838, 12)

In [22]:
sum(data['Metascore'].isna() + data['Revenue (Millions)'].isna())

162

In [23]:
data = data.dropna()

In [51]:
m = WordNetLemmatizer()
stops = stopwords.words('english')

In [104]:
def evaluateReg(data: pd.DataFrame, lemmatize: Callable,
                regressor: Callable = Ridge, par: float = 1) -> None:
    data['text'] = data.Description.apply(lemmatize)
    input_text = list(data.lemma.values)
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text)]
    model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)
    vectors = []
    for x in documents:
        vec = list(model[x.tags][0])
        vectors.append(vec)
    split_df = pd.DataFrame(vectors,
                        columns=['v1', 'v2', 'v3','v4',"v5"])

    result = data.join(split_df, how='left')
    data_sm = result[['Runtime (Minutes)',"Year",
                'Rating', 'Votes',
                'Revenue (Millions)','Metascore',"v1","v2","v3","v4","v5"]
              ]
    data_sm = data_sm.dropna()

    X = data_sm.drop(["Rating"],axis=1).values 
    y = data_sm['Rating'].values

    sc = StandardScaler()

    X_train, X_test, y_train, y_test = train_test_split(sc.fit_transform(X),
                                                        y, random_state=42)
    regressor = regressor(par) 

    regressor.fit(X_train, y_train)
    y_preds = regressor.predict(X_test)
    print(mean_absolute_error(y_test, y_preds))
    print(mean_squared_error(y_test, y_preds))

In [92]:
evaluateReg(data,
            lambda x: x.lower().split())

0.4228922602440821
0.3341609313662102


In [101]:
evaluateReg(data,
            lambda x: word_tokenize(x.lower()))

0.42290358676569445
0.3341647263289313


In [102]:
evaluateReg(data,
            lambda x: [
                m.lemmatize(y.strip(punctuation))
                for y in word_tokenize(x.lower()) if y not in stops
                ])

0.4228345299128619
0.3340934012201121


После лемматизации с удалением стоп-слов и нормальной токенизацией ничего особенно не поменялось. А жаль.

In [103]:
evaluateReg(data,
            lambda x: [
                m.lemmatize(y.strip(punctuation))
                for y in word_tokenize(x.lower()) if y not in stops
                ],
            0)

evaluateReg(data,
            lambda x: [
                m.lemmatize(y.strip(punctuation))
                for y in word_tokenize(x.lower()) if y not in stops
                ],
            2)

evaluateReg(data,
            lambda x: [
                m.lemmatize(y.strip(punctuation))
                for y in word_tokenize(x.lower()) if y not in stops
                ],
            3)

evaluateReg(data,
            lambda x: [
                m.lemmatize(y.strip(punctuation))
                for y in word_tokenize(x.lower()) if y not in stops
                ],
            4)

0.42283660376232546
0.33421041119834966


0.4229093013433594
0.33407104766730406


0.4229544087459793
0.3340518927182101


0.4229514577825
0.3339426846302254


После мены коэффициента регуляризации -- тоже.

In [105]:
evaluateReg(data,
            lambda x: [
                m.lemmatize(y.strip(punctuation))
                for y in word_tokenize(x.lower()) if y not in stops
                ],
            Lasso)

0.7520536127593316
0.8563704428453197


Лассо гораздо лучше справляется с задачей.

In [106]:
evaluateReg(data,
            lambda x: word_tokenize(x.lower()),
            Lasso)

0.7520536127593316
0.8563704428453197


Хотя на нормальную токенизацию ему тоже всё равно.